# Correlation Power Analysis Attack
## Hardware Security Project
### Master of Cybersecurity

Context...

## Dataset 1

In [ ]:
...

## Dataset 2

This time we are provided with:
- 16 trace files 
- 16 clock files
- 1 cleartext file

Steps:

What we tried

1.  We first tried to run the attack as we did in dataset1 to verify the real need of the clock files. As expected the key checksum wasn't matching with our results.

2. We plotted the clock files to try to see something but we couldn't see much (join graph).

3. We plotted the clock files along the trace files to try to see a correlation but we didn't identify any.

4. We tried to plot only some part of the traces along the corresponding clocks but we couldn't exploit any information since the traces are unaligned

5. We tried to align the traces by shifting each trace so that it's maximum aligns with the minimum index. After saving the aligned traces to files we ran the attack again as we did for dataset1 but the key checksum didn't match.

6. Finally we decided to look for flanks in the clock signals and capture the associated trace data.

How we achieved it

1. Data reading

- Initialize 2D numpy array for the cleartexts
    - Type: int
    - Dimmensions: 150 lines * 16 bytes
- Initilize a 3D numpy array to hold all the traces
    - Type: float
    - Dimmensions: 16 files * 150 lines * 50000 values
- Initilize a 3D numpy array to hold all the clocks
    - Type: float
    - Dimmensions: 16 files * 150 lines * 50000 values

2. Hamming Weights

...

3. Rising edge detections

The code is designed to detect rising edges (rising flanks) in a clock signal and extract a specific window of data from the trace signal associated with each detected edge

The first loop is iterating over all the 150 traces of a file. The second loop iterates over the 50,000 values of each trace.

After some test we defined a limit value after which the sample is considered as a flank.

- a value too high is not detecting any flank
- a value too low ...

To detect a flank we set the following conditions:

- The previous sample must be under the fixed limit
- The current sample must be above the fixed limit

If these condition are met we capture the trace samples arround the current clock peak (from j-5 to j+4) for further analysis